# Importing the Files

In [217]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import cluster
import seaborn as sns

In [3]:
df = pd.read_csv('movies.csv')

# Creating Dummy Variables for the Genres

In [5]:
all_genres = []
for x in df.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)    

In [18]:
zero_matrix = np.zeros((len(df), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns = genres)
gen = df.genres[0]
dummies.columns.get_indexer(gen.split('|'))

In [25]:
#Used part of the code written in the Wes McKiney Book pg 210. I had something similar, but this is much more efficient.
for i, gen in enumerate(df.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [26]:
movies_windic = df.join(dummies.add_prefix('Genre_'))

# Moving to Rec. System

In [33]:
#Assume UserID 1 wants a movie, WLOG as we can always change the initial value.

In [165]:
ratings = pd.read_csv('ratings.csv')
id1 = ratings[ratings.userId == 1]

In [166]:
#Let's calculate the average value for the person to give them better recs.
mean = id1.rating.mean()
overallrating = pd.DataFrame([np.zeros(20)], columns = movies_windic.columns.values[3:])
    

In [169]:
#Pretty much I added the a normalization to the genres to get the better genres that the person likes as y.
for x in id1.values:
    movie = movies_windic[movies_windic.movieId == x[1]].copy()
    movie = movie.values
    rating = x[2] - mean
    movierating = movie[0, 3:]
    movierating = movierating * rating
    overallrating+=movierating

In [170]:
#A normalization of the ratings to get try to get 1 to be the max
overallrating = overallrating / overallrating.values.max()
overallrating

,Genre_Adventure,Genre_Animation,Genre_Children,Genre_Comedy,Genre_Fantasy,Genre_Romance,Genre_Drama,Genre_Action,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Mystery,Genre_Sci-Fi,Genre_War,Genre_Musical,Genre_Documentary,Genre_IMAX,Genre_Western,Genre_Film-Noir,Genre_(no genres listed)
0,0.167574,0.845645,0.686625,-0.668351,-0.290435,-0.137636,1,-0.358476,-0.0439347,-1.09603,-1.37364,-0.324261,-0.510109,0.265163,0.625972,0,0,-0.0509331,0.057154,0


# Now we need to cluster to predict where overall ratings would go

In [212]:
y = movies_windic.copy()
y.rename(y.movieId, inplace = True)
y.drop('title', axis = 1, inplace = True)
y.drop('genres', axis = 1, inplace = True)
y.drop('movieId', axis = 1, inplace = True)

In [214]:
model = cluster.KMeans(n_clusters = 20)
predict = model.fit(y)
labels = predict.labels_

In [215]:
y['clusters'] = labels

In [226]:
id1pred = model.predict(overallrating)
print(id1pred)
y[y['clusters']== id1pred[0]]

[2]


,Genre_Adventure,Genre_Animation,Genre_Children,Genre_Comedy,Genre_Fantasy,Genre_Romance,Genre_Drama,Genre_Action,Genre_Crime,Genre_Thriller,...,Genre_Mystery,Genre_Sci-Fi,Genre_War,Genre_Musical,Genre_Documentary,Genre_IMAX,Genre_Western,Genre_Film-Noir,Genre_(no genres listed),clusters
14,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
24,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
26,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
27,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
31,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
34,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
40,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
43,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
53,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
55,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
